# RNN 활용 텍스트 분석

> imdb 영화 리뷰 데이터셋(영문)

## 텍스트 분석을 위해 개선된 알고리즘의 종류

### [1] RNN

- Embedding층은 단순하게 데이터의 표현을 학습하여 데이터 사전을 구축하는 것
- 하지만 유사한 의미의 단어를 비슷한 공간에 매핑할 수 있지만, 시퀀스 데이터의 중요한 특성인 순서와 맥락까지 고려한 것은 아니다.
- 순환 신경망은 이 문제를 해결하기 위해 고안된 층
- 컨볼루션 신경망의 반대되는 개념.

### [2] LSTM

- RNN의 그래디언트 손실문제를 보완한 방법
- 정보를 여러 시점에 걸쳐 나르는 장치(‘Cell state’)가 추가되었다.
- 이로 인해 그래디언트를 보존할 수 있어 그래디언트 손실 문제가 발생하지 않도록 도와준다.

### [3] GRU

- 게이트 메커니즘이 적용된 RNN의 일종으로 LSTM에서 영감을 받았으며 더 간략한 구조를 갖는다.
- 한국인 조경현 박사님이 제안한 방법

> 이전 예제에서 학습 모델을 정의과정의 알고리즘만 변경하면 되기 때문에 큰 맥락은 앞 예제와 동일하다. 학습 시간이 매우 오래 걸리는 예제이므로 가급적 GPU가 탑재된 컴퓨터에서 실습하는 것이 좋다.

## #01. 준비작업

### [1] 패키지 참조

In [ ]:
!pip uninstall -y hossam
!pip install --upgrade git+https://github.com/leekh4232/hossam-data-helper.git

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# 연결된 모듈이 업데이트 되면 즉시 자동 로드함
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings(action="ignore")

from hossam.util import *
from hossam.plot import *
from hossam.tensor import *

# 순수 Tensorflow 코드로 구현하고자 할 경우 아래 구문이 필요함
#from tensorflow.keras.layers import Dense, Embedding, Flatten, SimpleRNN, LSTM, GRU

### [2] 데이터 가져오기

Tensorflow에 내장된 데이터가 있지만, 동일 데이터를 미리 엑셀로 저장 두었다.

In [ ]:
origin = my_read_excel(
    "https://data.hossam.kr/tmdata/imdb.xlsx", categories=["sentiment"], info = False
)

## #02. 데이터 전처리

### [1] 종속변수 라벨링

In [ ]:
df = my_labelling(origin, "sentiment")
my_pretty_table(df.head(10))

### [2] 문자열 전처리

* 축약형 문자열 처리
* 이메일 주소 제거
* HTML 태그 제거
* URL 제거
* 숫자 제거
* 특수문자 제거
* 불용어 제거

#### (1) 영어 불용어 목록

In [ ]:
stopwords = my_stopwords("en")
stopwords[:10]

#### (2) 전처리 수행

In [ ]:
%%time

df2 = my_text_data_preprocessing(df, "review", rm_abbr=True, rm_email=True, rm_html=True, rm_url=True, rm_num = True, rm_special=True, stopwords=stopwords)
my_pretty_table(df2.head(10))

### [3] 문자열 토큰화 및 패딩

In [ ]:
%%time

df3, token_set, vocab_size = my_token_process(df2, xname='review', yname='sentiment', threshold=30, max_word_count=None)

print("vocab_size:", vocab_size)

max_word_count = df3['count'].max()
print("max_word_count:", max_word_count)

df3

## #04. 학습 모형 적합

### [2] 데이터 분할

In [ ]:
x_train, x_test, y_train, y_test = my_train_test_split(token_set, ydata=df3['sentiment'])

print(
    "훈련용 데이터셋 크기: %s, 검증용 데이터셋 크기: %s" % (x_train.shape, x_test.shape)
)
print("훈련용 레이블 크기: %s, 검증용 레이블 크기: %s" % (y_train.shape, y_test.shape))

### [3] 모델 생성

dropout은 지정된 비율만큼 학습을 건너뛰게 하는 파라미터. 이 파라미터를 사용하게 되면 과거 학습정보를 잃어버릴 확률이 높아지고 그에 따라 모델 성능이 나빠질 가능성이 있다.

recurrent_dropout(순환드롭아웃)은 과거 학습정보를 잃어버리는 문제를 해결하기 위해 적용하는 옵션.

In [ ]:
%%time

# model = Sequential()
# model.add(Embedding(input_dim = vocab_size, output_dim = 32, input_length = max_word_count))
# model.add(SimpleRNN(32, return_sequences=True, dropout=0.15, recurrent_dropout=0.15))
# model.add(SimpleRNN(16))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(optimizer='adam',loss = 'binary_crossentropy', metrics = ['acc'])
# model.summary()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = my_tf(
    x_train=x_train,
    y_train=y_train,
    x_test=x_test,
    y_test=y_test,
    layer=[
        dict(type="embedding", input_dim=vocab_size, output_dim=32, input_length=max_word_count),
        dict(type="rnn", units=32, return_sequences=True, dropout=0.15, recurrent_dropout=0.15),
        dict(type="rnn", units=16),
        dict(type="flatten"),
        dict(type="dense", units=1, activation="sigmoid")
    ],
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["acc"]
).to(device)

model.summary()

## #06. 학습 결과 적용

### [1] 검증 데이터에 대한 예측 결과 산정

In [ ]:
result = model.predict(x_test)
data_count, case_count = result.shape
print("%d개의 검증 데이터가 %d개의 경우의 수를 갖는다." % (data_count, case_count))
result

### [2] 예측 결과를 1차원 배열로 변환

In [ ]:
f_results = result.flatten()
f_results = np.round(f_results, 0)
f_results

### [3] 오차행렬


In [ ]:
my_confusion_matrix(y=y_test, y_pred=f_results, figsize=(7, 5))

## #07. 학습 결과 활용

### [1] 임의의 문장

In [ ]:
review = [
    "I had a lot of fun watching it.",
    "I didn't notice how time passed.",
    "I was happy the whole time I watched it.",
    "worst movie",
    "such a boring movie",
]

### [2] 문자열 전처리

In [ ]:
review_pre = [my_text_preprocessing(x, stopwords=stopwords) for x in review]
review_pre

### [3] 토큰화 + 패딩

In [ ]:
_, token_set, _ = my_token_process(review_pre, num_words=vocab_size, max_word_count=max_word_count)
token_set

### [5] 예측값 생성

In [ ]:
f_results = model.predict(token_set)
data_count, case_count = f_results.shape
print("%d개의 검증 데이터가 %d개의 경우의 수를 갖는다." % (data_count, case_count))
f_results

### [6] 결과 판정

In [ ]:
f_results = np.round(f_results.flatten(), 0)

for i, v in enumerate(review):
    k = "긍정" if f_results[i] == 1 else "부정"
    print(f"[{k}] {v}")